In [ ]:
!pip install telethon requests qrcode Pillow nest_asyncio opencv-python


In [11]:
# Установка необходимых библиотек
!pip install telethon requests nest_asyncio

# Импорт библиотек
import os
import json
import requests
import asyncio
from telethon import TelegramClient
import nest_asyncio

# Применение nest_asyncio для корректной работы
nest_asyncio.apply()

# --- Ваши данные из GitHub Secrets ---
api_id = int(os.environ['API_ID'])
api_hash = os.environ['API_HASH']
phone = os.environ['PHONE_NUMBER']
n8n_webhook_url = os.environ['N8N_WEBHOOK_URL']
# ---------------------------------------------

# --- Список каналов для парсинга ---
# Убедитесь, что каналы указаны правильно
channel_names = ['@BlackAudit', '@violavolkova']

async def main():
    # Подключение к Telegram
    client = TelegramClient('my_session', api_id, api_hash)
    await client.connect()

    # Проверка авторизации. Если сессия не найдена, скрипт завершится.
    if not await client.is_user_authorized():
        print("Ошибка: не удалось авторизоваться. Убедитесь, что файл my_session.session загружен в репозиторий.")
        return

    # Получение последних 5 сообщений из каждого канала в списке
    results = []
    for channel_name in channel_names:
        try:
            async for msg in client.iter_messages(channel_name, limit=5):
                if msg.text:
                    results.append({
                        'channel': channel_name,
                        'id': msg.id,
                        'text': msg.text,
                        'date': str(msg.date)
                    })
        except Exception as e:
            print(f"Ошибка при парсинге канала {channel_name}: {e}")
            continue

    # Отправка данных в n8n
    try:
        requests.post(n8n_webhook_url, json=results)
        print("Данные успешно отправлены в n8n.")
    except Exception as e:
        print(f"Ошибка при отправке данных в n8n: {e}")

    # Отключение от Telegram
    await client.disconnect()

if __name__ == '__main__':
    asyncio.run(main())

Данные успешно отправлены в n8n.
